# '사람인' 기업 정보 크롤링 - 헤드헌팅 분류

In [ ]:
#import package
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [ ]:
headers={'User-Agent' : ''}
url=''
driver=webdriver.Chrome()
driver.get(url)
req=requests.get(url, headers = headers)
content=req.content

time.sleep(0.5)
soup= BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
res=pd.DataFrame(columns=['공고명', '회사명', '채용정보-근무지', '채용정보-경력', '채용정보-학력', '채용정보-고용형태', '직무', '일자', '회사명2', '공고명2', '기업형태', '업종', '설립일', '매출액'])

title=[]
company=[]
place = []
lv= []
edu= []
category= []
job= []
day= []

name=[]
title2=[]
company2=[]
category=[]
day=[]
money=[]

# 19페이지까지 크롤링
for a in range(1,3):
    
    # 100개씩 크롤링
    for p in range (1, 101):
        print(f"Processing page {p}")  # 현재 페이지 번호 출력
        time.sleep(2)
        
        # (len(soup.select('div.cl_md'))) 길이만큼 반복
        for i in range (len(soup.select('div.job_sector'))):
            time.sleep(0.5)
        
        # 공고명
            title.append(soup.select('div.area_job')[i].select('a')[0].text)
        # 회사명
            company.append(re.sub('\s+', '', soup.select('div.area_corp')[i].select('strong')[0].text))
        # 채용정보- 근무지, 경력, 학력, 고용형태
        # 정보가 없는 경우(결측치) 빈 셀로 처리
            try:
                place.append(soup.select('div.job_condition')[i].select('span')[0].text)
            except IndexError:
                place.append(' ')

            lv.append(soup.select('div.job_condition')[i].select('span')[1].text)
            edu.append(soup.select('div.job_condition')[i].select('span')[2].text)

            try:
                category.append(soup.select('div.job_condition')[i].select('span')[3].text)
            except IndexError:
                category.append(' ')
                # 에러 확인 위해 print
                print(f"IndexError occurred at page {p}, index {i}")
        # 직무
            job.append(re.sub('\s+', ' ', soup.select('div.job_sector')[i].text).split(' 외 ')[0])
        # 일자
            day.append(soup.select('div.job_sector')[i].select('span.job_day')[0].text)
            
        try:
            
            # 개별 공고 클릭
            try:
                driver.find_element(By.XPATH, f'//*[@id="recruit_info_list"]/div[1]/div[{p}]/div[2]/h2/a').click() 
            except:
                driver.find_element(By.XPATH, f'//*[@id="recruit_info_list"]/div[1]/div[{p}]/div[1]/h2/a').click()
            time.sleep(2)
            driver.switch_to.window(driver.window_handles[-1])

            soup= BeautifulSoup(driver.page_source, 'html.parser')

            time.sleep(0.5)    

            # 기업명
            name.append(re.sub('\s+', '', soup.select('div.title_inner')[0].select('a')[0].text))
            # 구인글 제목
            title.append(re.sub('\s+', ' ', soup.select('h1.tit_job')[0].text))

            # 정보가 없는 경우 빈 셀로 처리   

            try:
                # 'div.info_area' 클래스 내부의 모든 <dl> 요소를 선택합니다.
                dl_elements = soup.select('div.info_area')[0].select('dl')

                # 추출된 정보를 저장할 변수를 초기화합니다.
                company_text, category_text, day_text, money_text = "", "", "", ""

                # 각 <dl> 요소에 대해 반복합니다.
                for dl_element in dl_elements:
                    # <dl> 요소의 텍스트 내용을 추출합니다.
                    dl_text = dl_element.text

                    # 회사 형태 정보가 있는지 확인합니다.
                    if '기업형태' in dl_text:
                        # 회사 형태를 추출하고 선행/후행 공백을 제거합니다.
                        company_text = dl_text.split('기업형태')[1].strip()

                    # 업종 정보가 있는지 확인합니다.
                    elif '업종' in dl_text:
                        # 업종을 추출하고 선행/후행 공백을 제거합니다.
                        category_text = dl_text.split('업종')[1].strip()

                    # 설립일 정보가 있는지 확인합니다.
                    elif '업력' in dl_text:
                        # 설립일을 추출하고 선행/후행 공백을 제거합니다.
                        day_text = re.sub('\s+', ' ', dl_text).split('설립일')[1].strip()

                    # 매출액 정보가 있는지 확인합니다.
                    elif '매출액' in dl_text:
                        # 매출액을 추출하고 선행/후행 공백을 제거합니다.
                        money_text = re.sub('\s+', ' ', dl_text).split('매출액')[1].strip()

                # 추출된 정보를 각각의 리스트에 추가합니다.
                # 만약 추출된 정보가 비어 있다면, 정렬을 유지하기 위해 빈 문자열을 추가합니다.
                company.append(company_text if company_text else '')
                category.append(category_text if category_text else '')
                day.append(day_text if day_text else '')
                money.append(money_text if money_text else '')

            except IndexError:
                # IndexError가 발생하는 경우 (예: 인덱스가 범위를 벗어난 경우), 정렬을 유지하기 위해 빈 문자열을 추가합니다.
                company.append('')
                category.append('')
                day.append('')
                money.append('')

            driver.close()

            time.sleep(0.5)
            driver.switch_to.window(driver.window_handles[-1]) 

            # 만약 행을 찾을 수 없다면, 더 이상 행이 없으므로 루프를 중지합니다.
            if not soup.select('h1.tit_job'):
                break
        # 에러 발생 시 중지.
        except NoSuchElementException:
            break
    
in_temp=pd.DataFrame([title, company, place, lv, edu, category, job, day, name, title2, company2, category, day, money], index=['공고명', '회사명', '채용정보-근무지', '채용정보-경력', '채용정보-학력', '채용정보-고용형태', '직무', '일자', '회사명2', '공고명2', '기업형태', '업종', '설립일', '매출액']).T
in_result2=pd.concat([res, in_temp], ignore_index=True)